In [ ]:
from functools import partial

import matplotlib.pyplot as plt

from superfv import (
    EulerSolver,
    plot_2d_slice,
)
from superfv.initial_conditions import decaying_isotropic_turbulence

In [ ]:
seed = 1
N = 64
T = [0.05, 0.01, 0.015, 0.02]

configs = {
    "p0": dict(riemann_solver="hllc", p=0),
    "MUSCL-Hancock": dict(
        riemann_solver="hllc",
        p=1,
        MUSCL=True,
        MUSCL_limiter="PP2D",
        flux_recipe=2,
    ),
    "ZS3": dict(
        riemann_solver="hllc",
        p=3,
        flux_recipe=2,
        lazy_primitives=True,
        ZS=True,
        GL=True,
        include_corners=True,
        PAD={"rho": (0, None)},
    ),
    # "MM3": dict(
    #     riemann_solver="hllc",
    #     p=3,
    #     flux_recipe=2,
    #     lazy_primitives=True,
    #     MOOD=True,
    #     limiting_vars=("rho", "vx"),
    #     cascade="muscl",
    #     MUSCL_limiter="moncen",
    #     max_MOOD_iters=1,
    #     NAD=True,
    #     NAD_rtol=1e-2,
    #     NAD_atol=1e-8,
    #     include_corners=True,
    #     PAD={"rho": (0, None)},
    # ),
}

In [ ]:
sims = {}
for name, config in configs.items():
    print(f"Running simulation: {name}")
    sim = EulerSolver(
        ic=partial(decaying_isotropic_turbulence, seed=seed, M=10),
        isothermal=True,
        nx=N,
        ny=N,
        **config,
    )

    try:
        sim.run(
            T,
            q_max=2,
            allow_overshoot=True,
            muscl_hancock=config.get("MUSCL", False),
            log_freq=20,
        )
        sims[name] = sim
    except RuntimeError as e:
        print(f"Simulation '{name}' failed: {e}")

In [ ]:
sim = sims["p0"]

fig, axs = plt.subplots(
    len(T) + 1, 2, sharex=True, sharey=True, figsize=(6, (len(T) + 1) * 2)
)

axs[0, 0].set_title(r"$\rho$")
axs[0, 1].set_title(r"$v$")

for i, t in enumerate([0] + T):
    axs[i, 0].set_ylabel(f"$t={t}$")
    plot_2d_slice(sim, axs[i, 0], "rho", cell_averaged=True, colorbar=True, t=t)
    plot_2d_slice(sim, axs[i, 1], "v", cell_averaged=True, colorbar=True, t=t)

In [ ]:
sim = sims["MUSCL-Hancock"]

fig, axs = plt.subplots(
    len(T) + 1, 2, sharex=True, sharey=True, figsize=(6, (len(T) + 1) * 2)
)

axs[0, 0].set_title(r"$\rho$")
axs[0, 1].set_title(r"$v$")

for i, t in enumerate([0] + T):
    axs[i, 0].set_ylabel(f"$t={t}$")
    plot_2d_slice(sim, axs[i, 0], "rho", cell_averaged=True, colorbar=True, t=t)
    plot_2d_slice(sim, axs[i, 1], "v", cell_averaged=True, colorbar=True, t=t)

In [ ]:
sim = sims["ZS3"]

fig, axs = plt.subplots(
    len(T) + 1, 2, sharex=True, sharey=True, figsize=(6, (len(T) + 1) * 2)
)

axs[0, 0].set_title(r"$\rho$")
axs[0, 1].set_title(r"$v$")

for i, t in enumerate([0] + T):
    axs[i, 0].set_ylabel(f"$t={t}$")
    plot_2d_slice(sim, axs[i, 0], "rho", cell_averaged=True, colorbar=True, t=t)
    plot_2d_slice(sim, axs[i, 1], "v", cell_averaged=True, colorbar=True, t=t)